# Random Forest

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [4]:
data_train = pd.read_csv("processed_training.csv")
data_test = pd.read_csv("processed_test.csv")

train_cols = data_train.columns.tolist()
data_test = data_test[train_cols].copy()

data_train.drop(['venue', 'start_hour'], axis=1, inplace=True)
data_test.drop(['venue', 'start_hour'], axis=1, inplace=True)

data_train = pd.get_dummies(data_train, columns=['team1_name', 'team2_name', 'season_type'])
data_test = pd.get_dummies(data_test, columns=['team1_name', 'team2_name', 'season_type'])

# ensure the train and test data have the same columns

train_cols = set(data_train.columns)
test_cols = set(data_test.columns)

print(train_cols - test_cols)
print(test_cols - train_cols)

from sklearn.utils import shuffle

data_train = shuffle(data_train)

Y_train = data_train['attendance'].to_numpy()
X_train = data_train.drop(['attendance'], axis=1).to_numpy()

Y_test = data_test['attendance'].to_numpy()
X_test = data_test.drop(['attendance'], axis=1).to_numpy()

# print shape of data
print("X_train: ", X_train.shape)
print("Y_train", Y_train.shape)
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

set()
set()
X_train:  (12270, 273)
Y_train (12270,)
X_test (2445, 273)
Y_test (2445,)


In [5]:
# use 5-folds cross validation to find the best alpha
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error

kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X_train)

5

In [11]:
parameters = {
    'n_estimators': [100],
    'max_features': ['sqrt'],
}

rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters, cv=kf, scoring='neg_mean_squared_error')
clf.fit(X_train, Y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)

c:\Users\user\Anaconda3\envs\python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\user\Anaconda3\envs\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\Anaconda3\envs\python39\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\user\Anaconda3\envs\python39\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
 

MemoryError: could not allocate 1368391680 bytes

In [ ]:
# use the best parameters to train the model
best_model = RandomForestClassifier(**clf.best_params_)
best_model.fit(X_train, Y_train)

RandomForestClassifier(max_depth=5, n_estimators=10)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

train_predict = best_model.predict(X_train)
print("Train MSE: ", mean_squared_error(Y_train, train_predict))
print("Train RMSE: ", np.sqrt(mean_squared_error(Y_train, train_predict)))
print("Train MAE: ", mean_absolute_error(Y_train, train_predict))
print("Train MAPE: ", mean_absolute_percentage_error(Y_train, train_predict))

Train MSE:  177535034.50635695
Train RMSE:  13324.227351195901
Train MAE:  10692.783129584352
Train MAPE:  0.41882512263086236
